In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

# Loading the dataset

In [2]:
iris = load_iris()

In [3]:
iris.data.shape, iris.target.shape, iris.target_names

((150, 4), (150,), array(['setosa', 'versicolor', 'virginica'], dtype='<U10'))

In [4]:
names = iris.target_names

#### - pre-processing

In [5]:
x_train, x_test, y_train, y_test= train_test_split(iris.data, iris.target)

In [6]:
for x in [x_train, y_train, x_test, y_test]:
    print (x.shape)

(112, 4)
(112,)
(38, 4)
(38,)


# Custom linear regression with torch.autograde and SGD optimizer

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parameter as parameter
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [8]:
class LRG(nn.Module):
    def __init__(self, x_n):
        super(LRG, self).__init__()
        self.x_n = x_n
        torch.manual_seed(10)
        self.w = parameter.Parameter(torch.rand(x_n, 3, requires_grad=True))
        self.b = parameter.Parameter(torch.zeros(1, 3, requires_grad=True))
        nn.init.xavier_uniform(self.w)
        self.params = nn.ParameterList([self.w, self.b])
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.parameters(), lr=0.01, )
    
    def forward(self, x):        
        return F.softmax(x.mm(self.w) + self.b)
    
    def epoch(self, x, labels):
        self.optimizer.zero_grad()
        pred = self(x)
        loss = self.criterion(pred, labels)
        loss.backward()
        self.optimizer.step()
        
    def train(self, epochs, x, labels):
        x = torch.Tensor(x)
        labels = torch.LongTensor(labels)
        for i in range(epochs):
            self.epoch(x, labels)
            
    def evaluate(self, x, labels):
        x = torch.Tensor(x)
        labels = torch.LongTensor(labels)
        preds = self(x)
        return (preds.argmax(dim=1) == labels).sum().item()/len(labels)

In [9]:
start = time.time()
lg = LRG(4)
lg.train(100, x_train, y_train)
print(time.time() - start)

0.09824657440185547


/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [10]:
lg.evaluate(x_train, y_train)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


0.3392857142857143

In [11]:
lg.evaluate(x_test, y_test)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


0.3157894736842105

# Building Custom optimizer

In [12]:
from torch.optim import Optimizer
from torch.optim.optimizer import required

In [13]:
class SGD(Optimizer):
    def __init__(self, params, lr=required):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
            
        defaults = dict(lr=lr)
        super(SGD, self).__init__(params, defaults)
        
    def __setstate__(self, state):
        super(SGD, self).__setstate__(state)

    def step(self):
        loss = None

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                p.data.add_(-group['lr'] * d_p)

        return loss

In [14]:
class LRG(nn.Module):
    def __init__(self, x_n):
        super(LRG, self).__init__()
        self.x_n = x_n
        torch.manual_seed(10)
        self.w = parameter.Parameter(torch.rand(x_n, 3, requires_grad=True))
        self.b = parameter.Parameter(torch.zeros(1, 3, requires_grad=True))
        nn.init.xavier_uniform(self.w)
        self.params = nn.ParameterList([self.w, self.b])
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = SGD(self.parameters(), lr=0.01)
    
    def forward(self, x):        
        return F.softmax(x.mm(self.w) + self.b)
    
    def epoch(self, x, labels):
        self.optimizer.zero_grad()
        pred = self(x)
        loss = self.criterion(pred, labels)
        loss.backward()
        self.optimizer.step()
        
    def train(self, epochs, x, labels):
        x = torch.Tensor(x)
        labels = torch.LongTensor(labels)
        for i in range(epochs):
            self.epoch(x, labels)
            
    def evaluate(self, x, labels):
        x = torch.Tensor(x)
        labels = torch.LongTensor(labels)
        preds = self(x)
        return (preds.argmax(dim=1) == labels).sum().item()/len(labels)

In [15]:
start = time.time()
lg = LRG(4)
lg.train(100, x_train, y_train)
print(time.time() - start)

0.0352325439453125


/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [16]:
lg.evaluate(x_test, y_test)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


0.3157894736842105

In [17]:
lg.evaluate(x_train, y_train)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


0.3392857142857143